# Interface-based State Transition Analysis and Rate calculation (iSTAR) Workflow

This notebook implements the iSTAR method for calculating crossing probabilities (Pcross) from transition interface sampling (TIS) simulations using infretis weights. The iSTAR approach builds a Markov state model at the interfaces to efficiently calculate transition probabilities.

## Key Features:
- **Weight Calculation**: Uses infretis weights for proper statistical weighting of paths
- **Transition Matrix Construction**: Builds transition matrices from interface-to-interface transition probabilities  
- **Global Pcross Calculation**: Computes the global crossing probability using the iSTAR Markov model
- **WHAM Integration**: Incorporates WHAM-based methods for improved statistical accuracy

## Workflow Overview:
1. Load and process infretis simulation data
2. Calculate path weights using infretis methodology
3. Compute transition probabilities between interfaces
4. Construct the iSTAR transition matrix
5. Calculate global crossing probability from the Markov model
6. Compare with traditional WHAM methods

In [86]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
from typing import Dict, List, Tuple, Optional
import warnings

# Inftools imports
import sys
sys.path.append('/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/APPTIS/inftools')

from inftools.tistools.path_weights import get_path_weights
from inftools.analysis.Wham_Pcross import run_analysis
from inftools.misc.tomlreader import infretis_data_reader

print("Libraries imported successfully!")
print("Available functions:")
print("- get_path_weights: Calculate unbiased weights for paths")
print("- run_analysis: WHAM-based crossing probability analysis")
print("- read_toml: Read TOML configuration files")

Libraries imported successfully!
Available functions:
- get_path_weights: Calculate unbiased weights for paths
- run_analysis: WHAM-based crossing probability analysis
- read_toml: Read TOML configuration files


In [87]:
def calculate_infretis_weights(data_file: str, toml_file: str, nskip: int = 0) -> Dict:
    """
    Calculate infretis weights for paths, based on the path_weights.py methodology.
    Updated to handle ptype information and derive direction from aXMYb format.
    
    Parameters:
    -----------
    data_file : str
        Path to infretis_data.txt file
    toml_file : str  
        Path to infretis.toml configuration file
    nskip : int
        Number of initial entries to skip
        
    Returns:
    --------
    Dict containing path data and weights
    """
    import tomli
    import re
    import random
    
    def parse_ptype_direction(ptype):
        """
        Parse ptype to extract direction based on interface indices.
        Uses the exact logic from conv_inf_py.py
        """
        # Handle simple ptype formats (ensemble 0)
        if ptype in ['RMR', 'RML', 'LMR', 'LML']:
            return 1
        
        # Handle complex ptype formats with interface indices
        # Pattern to match: digits + letters + digits
        match = re.match(r'^(\d+)([LR]M[LR])(\d+)$', ptype)
        if match:
            try:
                a = int(match.group(1))  # First interface index
                b = int(match.group(3))  # Last interface index
                
                if a < b:
                    return 1
                elif a > b:
                    return -1
                else:  # a == b
                    return random.choice([1, -1])
            except ValueError:
                # If parsing fails, default to 1
                return 1
        
        # Default case
        return 1

    def extract_ptype_middle(ptype):
        """
        Extract the middle part (XMX) from ptype.
        Uses the exact logic from conv_inf_py.py
        """
        # Handle simple ptype formats (ensemble 0)
        if ptype in ['RMR', 'RML', 'LMR', 'LML']:
            return ptype
        
        # Handle complex ptype formats with interface indices
        match = re.match(r'^(\d+)([LR]M[LR])(\d+)$', ptype)
        if match:
            return match.group(2)  # Return the middle part (XMX)
        
        # Default case - return as is
        return ptype
    
    # Load configuration
    with open(toml_file, "rb") as f:
        toml_config = tomli.load(f)
    interfaces = toml_config["simulation"]["interfaces"]
    
    # Load data
    data = np.loadtxt(data_file, dtype=str)
    data = data[nskip:]  # Skip initial entries
    
    # Check if we have ptype information (look for correct patterns)
    has_ptype = False
    ptype_col = None
    
    # Look for ptype patterns in the data - use correct patterns
    for col in range(data.shape[1]):
        sample_values = data[:10, col]  # Check first 10 rows
        for val in sample_values:
            if isinstance(val, str) and (re.match(r'\d+[LR]M[LR]\d+', val) or val in ['RMR', 'RML', 'LMR', 'LML']):
                has_ptype = True
                ptype_col = col
                break
        if has_ptype:
            break
    
    if has_ptype:
        print(f"Found ptype information in column {ptype_col}")
        # Extract direction information from ptype
        directions = []
        start_interfaces = []
        end_interfaces = []
        
        for i, row in enumerate(data):
            ptype = row[ptype_col]
            if isinstance(ptype, str):
                # Parse using the correct logic from conv_inf_py.py
                direction = parse_ptype_direction(ptype)
                directions.append(direction)
                
                # Extract start and end interface indices
                if ptype in ['RMR', 'RML', 'LMR', 'LML']:
                    # Simple format - ensemble 0
                    start_interfaces.append(0)
                    end_interfaces.append(0)
                else:
                    # Complex format with interface indices
                    match = re.match(r'^(\d+)([LR]M[LR])(\d+)$', ptype)
                    if match:
                        a = int(match.group(1))  # start interface index
                        b = int(match.group(3))  # end interface index
                        start_interfaces.append(a)
                        end_interfaces.append(b)
                    else:
                        start_interfaces.append(0)
                        end_interfaces.append(0)
            else:
                directions.append(0)  # Default for non-ptype entries
                start_interfaces.append(0)
                end_interfaces.append(0)
    else:
        print("No ptype information found, using standard format")
        directions = None
        start_interfaces = None
        end_interfaces = None
    
    # Identify non-zero paths (those with "----" in the zero-ensemble column)
    # Adjust column index based on whether ptype is present
    zero_col = 4 if not has_ptype else 5  # Assumes ptype is typically after maxop
    if zero_col < data.shape[1]:
        non_zero_paths = data[:, zero_col] == "----"
    else:
        # Fallback: look for "----" in any column after maxop
        non_zero_paths = data[:, 3] == "----"
    
    # Replace "----" with "0.0" for numerical processing
    data[data == "----"] = "0.0"
    
    # Extract path information
    D = {}
    D["pnr"] = data[non_zero_paths, 0:1].astype(int)  # Path numbers
    D["len"] = data[non_zero_paths, 1:2].astype(int)  # Path lengths
    D["maxop"] = data[non_zero_paths, 2:3].astype(float)  # Maximum order parameter
    
    # Add ptype-derived information if available
    if has_ptype:
        D["ptype"] = data[non_zero_paths, ptype_col]  # Path types
        D["direction"] = np.array([directions[i] for i in range(len(directions)) if non_zero_paths[i]])
        D["start_intf"] = np.array([start_interfaces[i] for i in range(len(start_interfaces)) if non_zero_paths[i]])
        D["end_intf"] = np.array([end_interfaces[i] for i in range(len(end_interfaces)) if non_zero_paths[i]])
    
    # Determine data columns for path_f and path_w
    data_start_col = ptype_col + 1 if has_ptype else 4
    D["path_f"] = data[non_zero_paths, data_start_col : data_start_col + len(interfaces)].astype(float)  # Path occurrences
    D["path_w"] = data[non_zero_paths, data_start_col + len(interfaces) : data_start_col + 2 * len(interfaces)].astype(float)  # Path weights
    
    # Calculate weights w = path_f / path_w
    w = D["path_f"] / D["path_w"]
    w[np.isnan(w)] = 0
    
    # Normalize weights to match total number of samples
    w = w / np.sum(w, axis=0) * np.sum(D["path_f"], axis=0)
    w[np.isnan(w)] = 0.0
    wsum = np.sum(w, axis=0)
    
    # # Calculate local crossing probabilities (ploc) using WHAM
    # ploc_wham = np.zeros(len(interfaces))
    # ploc_wham[0] = 1.0
    
    # for i, intf_p1 in enumerate(interfaces[1:]):
    #     h1 = D["maxop"] >= intf_p1
    #     nj = wsum[:i + 1]  # Number of paths crossing lambda_i for each ensemble up to i
    #     njl = np.sum(h1 * w[:, : i + 1], axis=0)  # Number of paths crossing lambda_i+1
    #     ploc_wham[i + 1] = np.sum(njl) / np.sum(nj / ploc_wham[: i + 1])
    
    # # Calculate unbiased path weights
    # A = np.zeros_like(D["maxop"])
    # Q = 1 / np.cumsum(wsum / ploc_wham[:-1])
    
    # for j, pathnr in enumerate(D["pnr"][:, 0]):
    #     # Find the highest interface crossed by this path
    #     K = min(
    #         np.where(D["maxop"][j] > interfaces)[0][-1] if np.any(D["maxop"][j] > interfaces) else 0, 
    #         len(interfaces) - 2
    #     )
    #     A[j] = Q[K] * np.sum(w[j])
    
    # Store results
    results = {
        'interfaces': interfaces,
        'path_data': D,
        'weights_matrix': w,
        # 'unbiased_weights': A,
        # 'ploc_wham': ploc_wham,
        # 'Q_factors': Q,
        'has_ptype': has_ptype
    }
    
    print(f"Processed {len(D['pnr'])} non-zero paths")
    print(f"Interfaces: {interfaces}")
    # print(f"Local crossing probabilities (WHAM): {ploc_wham}")
    
    if has_ptype:
        print(f"Path type information detected:")
        print(f"  Forward paths (dir=1): {np.sum(D['direction'] == 1)}")
        print(f"  Backward paths (dir=-1): {np.sum(D['direction'] == -1)}")
        print(f"  Other paths (dir=0): {np.sum(D['direction'] == 0)}")
    
    return results

print("Function calculate_infretis_weights updated with correct ptype handling from conv_inf_py.py!")

Function calculate_infretis_weights updated with correct ptype handling from conv_inf_py.py!


In [88]:
def compute_weight_matrices_weights(weight_results: Dict, tr: bool = False) -> Dict:
    """
    Compute 3D weight matrices from path data following original istar_analysis logic.
    
    This function constructs weight matrices [i,j,k] where:
    - i: ensemble index (path ensemble)
    - j: starting interface index
    - k: ending interface index
    
    Implements the original istar_analysis.py logic for:
    - tr (time reversal): boolean for applying time-reversal symmetry
    - Edge case handling for specific interface transitions
    - Proper direction mask logic and boundary conditions
    
    Parameters:
    -----------
    weight_results : Dict
        Results from calculate_infretis_weights function containing:
        - path_data (D): Dictionary with path information including path_f and path_w
        - interfaces: List of interface positions
        - has_ptype: Boolean indicating if ptype information is available
    tr : bool, optional
        If True, applies time-reversal symmetry by symmetrizing weight matrices.
        Default is False.
        
    Returns:
    --------
    Dict containing:
        - weight_matrix_3d: 3D array [i,j,k] with weights for ensemble i, from interface j to k
        - count_matrix_3d: 3D array [i,j,k] with path counts for ensemble i, from interface j to k
        - weight_matrix_2d: 2D array [j,k] with total weights (summed over ensembles)
        - count_matrix_2d: 2D array [j,k] with total count (summed over ensembles)
        - ensemble_totals: 1D array with total weights per ensemble
        - transition_summary: Dictionary with detailed transition statistics
        - tr_applied: Boolean indicating if time-reversal symmetry was applied
    """
    
    # Extract data from weight_results
    D = weight_results['path_data']
    interfaces = weight_results['interfaces']
    has_ptype = weight_results.get('has_ptype', False)
    
    n_interfaces = len(interfaces)
    n_ensembles = len(interfaces)  # Number of ensembles equals number of interfaces
    n_paths = len(D['pnr'])
    
    print(f"Computing weight matrices for {n_paths} paths")
    print(f"Structure: Dictionary of 2D matrices [ensemble_idx][start_interface, end_interface]")
    print(f"Dimensions: {n_ensembles} ensembles x {n_interfaces} interfaces x {n_interfaces} interfaces")
    print(f"Following original istar_analysis.py logic with tr={tr}")
    
    # Initialize dictionaries of 2D matrices {ensemble_i: [start_interface_j, end_interface_k]}
    weight_matrix_3d = {i: np.zeros((n_interfaces, n_interfaces)) for i in range(n_ensembles)}
    count_matrix_3d = {i: np.zeros((n_interfaces, n_interfaces)) for i in range(n_ensembles)}
    
    
    # Arrays to track totals
    ensemble_totals = np.zeros(n_ensembles)
    
    if has_ptype and 'start_intf' in D and 'end_intf' in D and 'direction' in D:
        print("Using ptype information with direction for istar_analysis-style computation")
        
        # Process each path
        for path_idx in range(n_paths):
            start_intf = int(D['start_intf'][path_idx])
            end_intf = int(D['end_intf'][path_idx])
            direction = int(D['direction'][path_idx])  # 1 for forward, -1 for backward, 0 for other
            
            # Validate interface indices
            if not (0 <= start_intf < n_interfaces and 0 <= end_intf < n_interfaces):
                continue
                
            # Process each ensemble for this path (following original istar_analysis logic)
            for i in range(n_ensembles):
                # Calculate weight as path_f / path_w for this ensemble
                path_f_k = D['path_f'][path_idx, i]
                path_w_k = D['path_w'][path_idx, i]
                
                if path_w_k != 0 and path_f_k != 0:  # Only process non-zero entries
                    weight = path_f_k / path_w_k
                    
                    # Apply original istar_analysis logic for j→k transitions
                    j, k = start_intf, end_intf
                    
                    # Determine if this path should be counted in ensemble i
                    should_count = False
                    
                    if j == k:
                        # Self-transitions: Special case for i==1 (ensemble 1) and j==0
                        if j == 0:
                            # Original logic: count LMR paths in ensemble 1 for 0→0 transitions
                            if 'LMR' in D['ptype'][path_idx] or ('LML' in D['ptype'][path_idx] and D['maxop'][path_idx] >= interfaces[1]):
                                k = 1  # Adjust to next interface for ensemble 1
                            should_count = True
                        elif j == len(interfaces) - 1:
                            k = len(interfaces) - 2  # Last interface self-transition
                            should_count = True  # Original logic: count RMR paths in last ensemble
                        else:
                            should_count = False
                            
                    elif j < k:
                        # Forward transitions (j → k where j < k)
                        
                        # Edge case 1: j==0 and k==1 (first interface to second)
                        if j == 0 and k == 1:
                            print("shouldnt happen first")
                            if i != 2:
                                # Use direction==1 for forward paths
                                should_count = (direction == 1)
                            elif i == 2:
                                # Special case: ensemble 2 uses different logic
                                # In original: dir_mask = masks[i]["LML"]
                                should_count = True  # Simplified - would need LML mask
                                
                        # Edge case 2: Last interface transition
                        elif j == len(interfaces)-2 and k == len(interfaces)-1:
                            print("shouldnt happen last")
                            # Original: dir_mask = masks[i]["RMR"]
                            should_count = True  # Simplified - would need RMR mask
                            
                        else:
                            # Standard forward transitions
                            should_count = (direction == 1)
                            
                    else:
                        # Backward transitions (j → k where j > k)
                        
                        # Edge case 1: j==1 and k==0 (second interface to first)
                        if j == 1 and k == 0:
                            print("shouldnt happen first backward")
                            if i != 2:
                                # Use direction==-1 for backward paths
                                should_count = (direction == -1)
                            elif i == 2:
                                # Special case: ensemble 2 uses different logic
                                should_count = True  # Simplified - would need LML mask
                                
                        # Edge case 2: Last interface backward transition
                        elif j == len(interfaces)-1 and k == len(interfaces)-2:
                            print("shouldnt happen last backward")
                            # Original: dir_mask = masks[i]["RMR"]
                            should_count = True  # Simplified - would need RMR mask
                            
                        else:
                            # Standard backward transitions
                            should_count = (direction == -1)
                    
                    # Count the transition if criteria are met
                    if should_count:
                        weight_matrix_3d[i][j, k] += weight
                        count_matrix_3d[i][j, k] += 1
                        ensemble_totals[i] += weight
    
    else:
        print("No ptype information with direction available")
        raise ValueError("Path data must contain 'start_intf', 'end_intf', and 'direction' for istar_analysis-style computation.")
    
    # Apply time-reversal symmetry if requested (following original istar_analysis logic)
    tr_applied = False
    if tr:
        tr_applied = True
        print("Applying time-reversal symmetry (tr=True)")
        
        for i in range(n_ensembles):
            # Original edge case logic for time reversal
            if i == 2 and weight_matrix_3d[i][1, 0] == 0:
                # In [1*] all LML paths are classified as 1 → 0 (for now).
                # Time reversal needs to be adjusted to compensate for this
                weight_matrix_3d[i][0, 1] *= 2
                print(f"  Applied tr edge case for ensemble 2: doubled weight_matrix_3d[{i}, 0, 1]")
                
            elif i == len(interfaces)-1 and weight_matrix_3d[i][-2, -1] == 0:
                weight_matrix_3d[i][-1, -2] *= 2
                print(f"  Applied tr edge case for last ensemble: doubled weight_matrix_3d[{i}, -1, -2]")
            
            # Properly symmetrize the matrix: X[i] = (X[i] + X[i].T) / 2.0
            weight_matrix_3d[i] = (weight_matrix_3d[i] + weight_matrix_3d[i].T) / 2.0
            count_matrix_3d[i] = (count_matrix_3d[i] + count_matrix_3d[i].T) / 2.0
    
    # Calculate 2D matrices by summing over ensembles
    weight_matrix_2d = np.zeros((n_interfaces, n_interfaces))
    count_matrix_2d = np.zeros((n_interfaces, n_interfaces))

    for i in range(n_ensembles):
        weight_matrix_2d += weight_matrix_3d[i]
        count_matrix_2d += count_matrix_3d[i]
    
    # Create transition summary
    total_weight = sum(np.sum(weight_matrix_3d[i]) for i in range(n_ensembles))
    total_transitions = sum(np.sum(count_matrix_3d[i]) for i in range(n_ensembles))
    
    # Analyze transition types across all ensembles
    forward_transitions = 0
    backward_transitions = 0
    self_transitions = 0
    forward_weight = 0
    backward_weight = 0
    self_weight = 0
    
    for i in range(n_ensembles):
        for j in range(n_interfaces):
            for k in range(n_interfaces):
                weight_ijk = weight_matrix_3d[i][j, k]
                count_ijk = count_matrix_3d[i][j, k]
                
                if count_ijk > 0:
                    if j < k:  # Forward transition
                        forward_transitions += count_ijk
                        forward_weight += weight_ijk
                    elif j > k:  # Backward transition
                        backward_transitions += count_ijk
                        backward_weight += weight_ijk
                    else:  # Self transition
                        self_transitions += count_ijk
                        self_weight += weight_ijk
    
    transition_summary = {
        'total_weight': total_weight,
        'total_transitions': total_transitions,
        'forward_transitions': forward_transitions,
        'backward_transitions': backward_transitions,
        'self_transitions': self_transitions,
        'forward_weight': forward_weight,
        'backward_weight': backward_weight,
        'self_weight': self_weight,
        'forward_weight_fraction': forward_weight / total_weight if total_weight > 0 else 0,
        'backward_weight_fraction': backward_weight / total_weight if total_weight > 0 else 0,
        'self_weight_fraction': self_weight / total_weight if total_weight > 0 else 0
    }
    
    # Print detailed results following original istar_analysis style
    print(f"\n=== 3D WEIGHT MATRICES RESULTS (istar_analysis style) ===")
    print(f"3D Matrix dimensions: {n_ensembles} x {n_interfaces} x {n_interfaces}")
    print(f"Total weight processed: {total_weight:.6f}")
    print(f"Total transitions: {total_transitions}")
    print(f"Non-zero 3D matrix elements: {sum(np.count_nonzero(weight_matrix_3d[i]) for i in range(n_ensembles))}")
    print(f"Time-reversal symmetry applied: {tr_applied}")
    
    # Print ensemble weights (like original "Sum weights ensemble i")
    print(f"\nEnsemble weight totals:")
    for i in range(n_ensembles):
        ensemble_sum = np.sum(weight_matrix_3d[i])
        print(f"  Sum weights ensemble {i}: {ensemble_sum:.4f}")
    
    print(f"\n2D Weight Matrix [start_interface, end_interface] (summed over ensembles):")
    print("Rows = start interface, Columns = end interface")
    for j in range(n_interfaces):
        row_str = f"Interface {j}: "
        for k in range(n_interfaces):
            row_str += f"{weight_matrix_2d[j, k]:8.4f} "
        print(row_str)
    
    print(f"\nTransition Analysis:")
    print(f"Forward transitions (j<k):  {forward_transitions:4f} paths, {forward_weight:8.4f} weight ({transition_summary['forward_weight_fraction']:.1%})")
    print(f"Backward transitions (j>k): {backward_transitions:4f} paths, {backward_weight:8.4f} weight ({transition_summary['backward_weight_fraction']:.1%})")
    print(f"Self transitions (j=k):     {self_transitions:4f} paths, {self_weight:8.4f} weight ({transition_summary['self_weight_fraction']:.1%})")

    # Show some 3D matrix details for non-zero entries
    print(f"\nNon-zero 3D matrix entries (first 10):")
    count = 0
    for i in range(n_ensembles):
        for j in range(n_interfaces):
            for k in range(n_interfaces):
                if weight_matrix_3d[i][j, k] > 0 and count < 10:
                    print(f"  weights[{i},{j},{k}] = {weight_matrix_3d[i][j, k]:.6f} (count: {count_matrix_3d[i][j, k]:.1f})")
                    count += 1
                if count >= 10:
                    break
            if count >= 10:
                break
        if count >= 10:
            break
    
    # Store and return results
    results = {
        'weight_matrix_3d': weight_matrix_3d,
        'count_matrix_3d': count_matrix_3d,
        'weight_matrix_2d': weight_matrix_2d,
        'count_matrix_2d': count_matrix_2d,
        'ensemble_totals': ensemble_totals,
        'transition_summary': transition_summary,
        'tr_applied': tr_applied,
        'interfaces': interfaces,
        'n_interfaces': n_interfaces,
        'n_ensembles': n_ensembles,
        'total_paths_processed': n_paths
    }
    
    return results

print("Function compute_weight_matrices_weights updated with original istar_analysis.py logic!")

Function compute_weight_matrices_weights updated with original istar_analysis.py logic!


In [89]:
def construct_istar_transition_matrix(P: np.ndarray, N: int) -> np.ndarray:
    """
    Construct the iSTAR transition matrix M from interface-to-interface transition probabilities.
    
    This implements the iSTAR model structure where states are defined at each interface
    with special handling for boundary states.
    
    Parameters:
    -----------
    P : np.ndarray
        Array of probabilities for paths between interfaces. P[i,j] represents the 
        probability of a path transitioning from interface i to interface j.
    N : int
        Number of interfaces in the system.
    
    Returns:
    --------
    np.ndarray
        Transition matrix M for the iSTAR model with dimensions (2*N, 2*N).
    """
    NS = 2 * N  # Dimension of the Markov state model
    
    # Validate input dimensions
    if P.shape[0] != N or P.shape[1] != N:
        raise ValueError(f"P matrix dimensions {P.shape} don't match N={N}")
    
    # Construct transition matrix
    M = np.zeros((NS, NS))
    
    # States [0-] and [0*+-] - special boundary handling
    M[0, 2] = 1            # Transition from state 0 to state 2
    M[2, 0] = P[0, 0]      # Transition from state 2 to state 0
    M[2, N+1:] = P[0, 1:]  # Transitions from state 2 to states N+1 and beyond
    M[1, 0] = 1            # Transition from state 1 to state 0
    M[-1, 0] = 1           # Transition from last state to state 0
    
    # Transitions from intermediate states to boundary
    if N > 1:
        M[N+1:-1, 1] = P[1:-1, 0]  # Transitions to state 1
    
    # Set up transitions for other interfaces
    for i in range(1, N-1):
        # Forward transitions
        if i < N-1:
            M[N+i, N+i+1:] = P[i, i+1:]
        # Backward transitions
        if i > 0:
            M[N+i, N+1:N+i] = P[i, 1:i]
    
    # Handle the last interface (special case)
    if N > 1:
        M[N, -1] = P[N-1, N-1] if N-1 < P.shape[1] else 0  # Self-transition or to absorbing state
    
    # Ensure matrix is properly stochastic (rows sum to 1)
    for i in range(NS):
        row_sum = np.sum(M[i, :])
        if row_sum == 0:
            M[i, i] = 1  # Set diagonal entry to 1 for rows that sum to zero
        elif row_sum > 0 and not np.isclose(row_sum, 1.0):
            M[i, :] /= row_sum  # Normalize to ensure stochastic matrix
    
    print(f"Constructed iSTAR transition matrix with shape {M.shape}")
    print(f"Matrix properties:")
    print(f"- Row sums: {[np.sum(M[i, :]) for i in range(min(5, NS))]}")
    print(f"- Non-zero entries: {np.count_nonzero(M)}/{NS*NS}")
    
    return M

def global_pcross_istar(M: np.ndarray, doprint: bool = False) -> Tuple:
    """
    Calculate global crossing probabilities from a transition matrix using the iSTAR approach.
    
    This function computes the probability to arrive at state -1 before reaching state 0,
    given that we start at state 0 and leave it.
    
    Parameters:
    -----------
    M : np.ndarray
        Transition matrix representing the Markov state model
    doprint : bool
        If True, prints detailed calculation information
        
    Returns:
    --------
    tuple
        (z1, z2, y1, y2) - solution vectors from the iSTAR calculation
    """
    NS = len(M)
    if NS <= 2:
        print("Warning: Matrix too small for meaningful iSTAR analysis")
        return None, None, None, None
    
    # Extract submatrices from the transition matrix
    Mp = M[2:-1, 2:-1]  # Transition probabilities between intermediate states
    a = np.identity(NS-3) - Mp  # I - Mp, used for solving linear system
    
    # Extract other submatrices from the transition matrix
    D = M[2:-1, np.array([0, -1])]  # Transitions from intermediate states to boundary states
    E = M[np.array([0, -1]), 2:-1]  # Transitions from boundary states to intermediate states
    M11 = M[np.array([0, -1]), np.array([0, -1])]  # Transitions between boundary states
    
    # Compute Z vector - solve the linear system
    z1 = np.array([[0], [1]])  # Initial condition
    try:
        z2 = np.linalg.solve(a, np.dot(D, z1))  # Solve (I-Mp)z2 = D·z1
    except np.linalg.LinAlgError:
        print("Warning: Singular matrix encountered in iSTAR calculation")
        return None, None, None, None
    
    # Compute H vector - final probability distribution
    y1 = np.dot(M11, z1) + np.dot(E, z2)  # Transitions to boundary states
    y2 = np.dot(D, z1) + np.dot(Mp, z2)   # Transitions to intermediate states
    
    if doprint:
        print("iSTAR calculation details:")
        print(f"Intermediate states matrix Mp shape: {Mp.shape}")
        print(f"Boundary transition matrix D shape: {D.shape}")
        print(f"z1 (boundary conditions): {z1.flatten()}")
        print(f"z2 (intermediate solutions): {z2.flatten()}")
        print(f"y1 (boundary probabilities): {y1.flatten()}")
        print(f"Global crossing probability: {y1[1, 0] if y1.shape[0] > 1 else 'N/A'}")
    
    return z1, z2, y1, y2

print("iSTAR matrix construction functions defined successfully!")

iSTAR matrix construction functions defined successfully!


In [90]:
def calculate_binless_pcross(weight_results: Dict) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculate binless crossing probability from infretis weights.
    
    This is based on the method in path_weights.py and provides a direct
    calculation of the crossing probability curve.
    
    Parameters:
    -----------
    weight_results : Dict
        Results from calculate_infretis_weights function
        
    Returns:
    --------
    tuple
        (order_params, pcross_values) - arrays for plotting/analysis
    """
    D = weight_results['path_data']
    A = weight_results['unbiased_weights']
    interfaces = weight_results['interfaces']
    
    # Sort paths by maximum order parameter
    idx = np.argsort(D["maxop"].flatten())
    maxop_sorted = D["maxop"][idx].flatten()
    weight_sorted = A[idx].flatten()
    sumw = np.sum(weight_sorted)
    
    # Calculate crossing probability
    res_y = [1.0]
    res_x = [interfaces[0]]
    
    for i, moi in enumerate(maxop_sorted):
        # Skip duplicate order parameter values
        if res_x[-1] == moi:
            continue
        # Crossing probability = sum of weights of paths that cross this point / total weight
        res_y.append(np.sum(weight_sorted[i:]) / sumw)
        res_x.append(moi)
    
    res_x = np.array(res_x)
    res_y = np.array(res_y)
    
    print(f"Calculated binless crossing probability with {len(res_x)} points")
    print(f"Order parameter range: {res_x.min():.3f} to {res_x.max():.3f}")
    print(f"Crossing probability range: {res_y.min():.6f} to {res_y.max():.6f}")
    
    return res_x, res_y

def plot_pcross_comparison(results_dict: Dict, title: str = "Crossing Probability Comparison"):
    """
    Plot comparison of different crossing probability calculations.
    
    Parameters:
    -----------
    results_dict : Dict
        Dictionary containing different pcross calculations with keys:
        - 'binless': (x, y) tuple from binless calculation
        - 'istar': global crossing probability from iSTAR
        - 'interfaces': interface positions
    """
    plt.figure(figsize=(12, 8))
    
    # Plot binless crossing probability
    if 'binless' in results_dict:
        x_binless, y_binless = results_dict['binless']
        plt.semilogy(x_binless, y_binless, 'b-', linewidth=2, label='Binless (infretis weights)')
    
    # Plot interfaces
    if 'interfaces' in results_dict:
        interfaces = results_dict['interfaces']
        for i, intf in enumerate(interfaces):
            plt.axvline(intf, color='black', linestyle='--', alpha=0.7, linewidth=1)
            plt.text(intf, 0.5, f'λ_{i}', rotation=90, verticalalignment='center')
    
    # Add iSTAR global crossing probability if available
    if 'istar_global' in results_dict:
        istar_pcross = results_dict['istar_global']
        plt.axhline(istar_pcross, color='red', linestyle='-', linewidth=2, 
                   label=f'iSTAR Global P_cross = {istar_pcross:.6f}')
    
    plt.xlabel('Order Parameter (λ)')
    plt.ylabel('Crossing Probability P(λ)')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(1e-6, 1.1)
    plt.tight_layout()
    plt.show()

print("Binless crossing probability and plotting functions defined successfully!")

Binless crossing probability and plotting functions defined successfully!


## Data Loading and Configuration

In this section, we'll load the infretis simulation data and configure the analysis parameters. The example uses the test data from the double-well system.

In [91]:
def create_test_data_with_ptype(filename: str, n_paths: int = 50, n_interfaces: int = 4):
    """
    Create test infretis_data.txt with ptype information for demonstration.
    
    This generates synthetic data with the correct aXMXb ptype format where:
    - a = start interface index
    - b = end interface index  
    - X can be L or R (representing left/right movement)
    - direction = 1 if a < b, -1 if a > b, random if a == b
    
    Parameters:
    -----------
    filename : str
        Output filename for the test data
    n_paths : int
        Number of paths to generate
    n_interfaces : int
        Number of interfaces
    """
    import random
    
    interfaces = [-1.0, -0.5, 0.0, 0.5, 1.0][:n_interfaces+1]  # Example interface positions
    
    with open(filename, 'w') as f:
        # Write header
        f.write("# Test data with ptype information\n")
        f.write("# path_nr\tlength\tmaxop\tptype\t")
        f.write("\t".join([f"ens{i:03d}" for i in range(n_interfaces)]))
        f.write("\t")
        f.write("\t".join([f"wgt{i:03d}" for i in range(n_interfaces)]))
        f.write("\n")
        
        # Generate synthetic paths
        for path_nr in range(n_paths):
            # Random path properties
            length = random.randint(10, 100)
            
            # Random start and end interfaces
            start_intf = random.randint(0, n_interfaces-1)
            end_intf = random.randint(0, n_interfaces-1)
            
            # Create ptype in correct aXMXb format from conv_inf_py.py
            if start_intf == 0 and end_intf == 0:
                # Simple format for ensemble 0
                ptype = random.choice(['RMR', 'RML', 'LMR', 'LML'])
            else:
                # Complex format with interface indices
                # Choose random middle part
                middle_parts = ['LMR', 'RML', 'LML', 'RMR']
                middle = random.choice(middle_parts)
                ptype = f"{start_intf}{middle}{end_intf}"
            
            # Maximum order parameter should be consistent with the path
            if end_intf >= start_intf:  # Forward path
                maxop = random.uniform(interfaces[start_intf], interfaces[end_intf+1] if end_intf < len(interfaces)-1 else 1.5)
            else:  # Backward path
                maxop = random.uniform(interfaces[end_intf], interfaces[start_intf+1] if start_intf < len(interfaces)-1 else 1.5)
            
            # Generate ensemble data (path_f and path_w)
            path_f = []
            path_w = []
            
            for i in range(n_interfaces):
                if i == start_intf:
                    # Path contributes to starting ensemble
                    f_val = random.uniform(0.5, 2.0)
                    w_val = random.uniform(0.5, 2.0)
                    path_f.append(f_val)
                    path_w.append(w_val)
                else:
                    # No contribution to other ensembles
                    path_f.append("----")
                    path_w.append("----")
            
            # Write path data
            f.write(f"{path_nr}\t{length}\t{maxop:.5f}\t{ptype}\t")
            f.write("\t".join([str(x) for x in path_f]))
            f.write("\t")
            f.write("\t".join([str(x) for x in path_w]))
            f.write("\n")
    
    print(f"Created test data file: {filename}")
    print(f"Generated {n_paths} paths with {n_interfaces} interfaces")
    print(f"File includes ptype information in correct aXMXb format")

# Test if we want to demonstrate with synthetic data
def test_ptype_format():
    """Test the ptype format parsing with synthetic data"""
    test_file = "test_infretis_data_with_ptype.txt"
    create_test_data_with_ptype(test_file, n_paths=20, n_interfaces=3)
    
    # Show first few lines
    with open(test_file, 'r') as f:
        lines = f.readlines()
    
    print("\nFirst few lines of generated test data:")
    for i, line in enumerate(lines[:8]):
        print(f"{i+1}: {line.strip()}")
    
    return test_file

print("Test data generation functions updated with correct ptype format!")

Test data generation functions updated with correct ptype format!


In [92]:
# Configuration for the analysis
DATA_DIR = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/APPTIS/inftools/test/test_staple"
TOML_FILE = f"{DATA_DIR}/infretis.toml"
DATA_FILE = f"{DATA_DIR}/infretis_data.txt"

# Analysis parameters
NSKIP = 0  # Number of initial entries to skip
VERBOSE = True
USE_TEST_PTYPE_DATA = False  # Set to True to demonstrate ptype functionality

print("Configuration:")
print(f"Data directory: {DATA_DIR}")
print(f"TOML file: {TOML_FILE}")
print(f"Data file: {DATA_FILE}")

# Check if files exist
import os
if os.path.exists(TOML_FILE):
    print("✓ TOML file found")
else:
    print("✗ TOML file not found")
    
if os.path.exists(DATA_FILE):
    print("✓ Data file found")
    # Show basic info about the data file
    with open(DATA_FILE, 'r') as f:
        lines = f.readlines()
    print(f"  - Total lines: {len(lines)}")
    print(f"  - First few lines:")
    for i, line in enumerate(lines[:5]):
        print(f"    {i+1}: {line.strip()}")
else:
    print("✗ Data file not found")

# Option to demonstrate ptype functionality
if USE_TEST_PTYPE_DATA:
    print(f"\n{'='*50}")
    print("DEMONSTRATING PTYPE FUNCTIONALITY")
    print(f"{'='*50}")
    
    # Create test data with ptype information
    test_data_file = test_ptype_format()
    
    # Create a simple TOML config for the test
    test_toml_content = '''
    [simulation]
    interfaces = [-1.0, -0.5, 0.0, 0.5]
    '''
    
    test_toml_file = "test_infretis.toml"
    with open(test_toml_file, 'w') as f:
        f.write(test_toml_content)
    
    print(f"\\nSwitching to test data for ptype demonstration:")
    print(f"Test TOML file: {test_toml_file}")
    print(f"Test data file: {test_data_file}")
    
    # Override the configuration
    TOML_FILE = test_toml_file
    DATA_FILE = test_data_file
    
    print("\\n✓ Test configuration ready for ptype demonstration")
else:
    print(f"\\nUsing standard data format (no ptype information)")
    print(f"Set USE_TEST_PTYPE_DATA = True to demonstrate ptype functionality")

Configuration:
Data directory: /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/APPTIS/inftools/test/test_staple
TOML file: /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/APPTIS/inftools/test/test_staple/infretis.toml
Data file: /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/APPTIS/inftools/test/test_staple/infretis_data.txt
✓ TOML file found
✓ Data file found
  - Total lines: 15126
  - First few lines:
    1: # ==========================================================================
    2: # 	xxx	len	max OP		000	001	002	003	004
    3: # ==========================================================================
    4: 3	 1170	 0.30010	-0.10011	0LMR4	----	----	----	----	----	----	----	----	----	----
    5: 0	 3021	-0.09986	-0.36087	RMR	1.0	----	----	----	----	1.0	----	----	----	----
\nUsing standard data format (no ptype information)
Set USE_TEST_PTYPE_DATA = True to demonstrate ptype functionality


## Step 1: Calculate Infretis Weights

First, we calculate the unbiased weights for all paths using the infretis methodology. This is the key improvement over the old workflow - using proper infretis weights instead of simple path counts.

In [93]:
# Demonstrate format detection and ptype parsing
print("=== DATA FORMAT ANALYSIS ===")

# Load and analyze the data format
try:
    data_sample = np.loadtxt(DATA_FILE, dtype=str, max_rows=10)
    print(f"Data shape: {data_sample.shape}")
    print(f"Number of columns: {data_sample.shape[1]}")
    
    # Check for ptype patterns - correct pattern from conv_inf_py.py
    import re
    ptype_found = False
    ptype_examples = []
    
    for col in range(data_sample.shape[1]):
        for row in range(data_sample.shape[0]):
            val = data_sample[row, col]
            # Look for patterns like "0LMR4", "10RML12", or simple "RMR", "LML" etc.
            if isinstance(val, str) and (re.match(r'\d+[LR]M[LR]\d+', val) or val in ['RMR', 'RML', 'LMR', 'LML']):
                ptype_found = True
                ptype_examples.append(val)
                print(f"Found ptype pattern '{val}' in column {col}, row {row}")
                
                # Parse and show direction using the correct logic from conv_inf_py.py
                if val in ['RMR', 'RML', 'LMR', 'LML']:
                    # Simple format (ensemble 0)
                    direction = 1
                    print(f"  -> Simple ptype format, Direction: {direction}")
                else:
                    # Complex format with interface indices
                    match = re.match(r'^(\d+)([LR]M[LR])(\d+)$', val)
                    if match:
                        a = int(match.group(1))  # First interface index
                        b = int(match.group(3))  # Last interface index
                        middle_part = match.group(2)  # XMX part
                        
                        if a < b:
                            direction = 1
                        elif a > b:
                            direction = -1
                        else:  # a == b
                            direction = 1  # or random choice as in conv file
                        
                        print(f"  -> Start interface: {a}, End interface: {b}, Middle: {middle_part}, Direction: {direction}")
    
    if not ptype_found:
        print("No ptype patterns (aXMXb format) detected in the data")
        print("This data uses the standard infretis format")
    else:
        print(f"\nPtype examples found: {list(set(ptype_examples))}")
    
    print(f"\nData preview (first 3 rows):")
    for i in range(min(3, data_sample.shape[0])):
        print(f"Row {i}: {data_sample[i]}")

except Exception as e:
    print(f"Error analyzing data format: {e}")

print(f"\n{'='*50}")

=== DATA FORMAT ANALYSIS ===
Data shape: (10, 15)
Number of columns: 15
Found ptype pattern '0LMR4' in column 4, row 0
  -> Start interface: 0, End interface: 4, Middle: LMR, Direction: 1
Found ptype pattern 'RMR' in column 4, row 1
  -> Simple ptype format, Direction: 1
Found ptype pattern '0LMR0' in column 4, row 2
  -> Start interface: 0, End interface: 0, Middle: LMR, Direction: 1
Found ptype pattern '0LMR4' in column 4, row 3
  -> Start interface: 0, End interface: 4, Middle: LMR, Direction: 1
Found ptype pattern '4RML0' in column 4, row 4
  -> Start interface: 4, End interface: 0, Middle: RML, Direction: -1
Found ptype pattern '0LML0' in column 4, row 5
  -> Start interface: 0, End interface: 0, Middle: LML, Direction: 1
Found ptype pattern 'RMR' in column 4, row 6
  -> Simple ptype format, Direction: 1
Found ptype pattern '0LMR4' in column 4, row 7
  -> Start interface: 0, End interface: 4, Middle: LMR, Direction: 1
Found ptype pattern 'RMR' in column 4, row 8
  -> Simple ptype 

/tmp/ipykernel_184690/380331464.py:6: UserWarning: Input line 1 contained no data and will not be counted towards `max_rows=10`.  This differs from the behaviour in NumPy <=1.22 which counted lines rather than rows.  If desired, the previous behaviour can be achieved by using `itertools.islice`.
Please see the 1.23 release notes for an example on how to do this.  If you wish to ignore this warning, use `warnings.filterwarnings`.  This warning is expected to be removed in the future and is given only once per `loadtxt` call.
  data_sample = np.loadtxt(DATA_FILE, dtype=str, max_rows=10)


In [94]:
# Calculate infretis weights
print("=== STEP 1: CALCULATING INFRETIS WEIGHTS ===")
weight_results = calculate_infretis_weights(DATA_FILE, TOML_FILE, nskip=NSKIP)


# Display key results
print("\nWeight calculation summary:")
print(f"Number of interfaces: {len(weight_results['interfaces'])}")
print(f"Interfaces: {weight_results['interfaces']}")
print(f"Number of paths processed: {len(weight_results['path_data']['pnr'])}")
# print(f"Local crossing probabilities (WHAM): {weight_results['ploc_wham']}")

# Show some statistics about the weights
# unbiased_weights = weight_results['unbiased_weights']
# print(f"\nWeight statistics:")
# print(f"Total weight: {np.sum(unbiased_weights):.6f}")
# print(f"Average weight: {np.mean(unbiased_weights):.6f}")
# print(f"Weight range: {np.min(unbiased_weights):.6f} to {np.max(unbiased_weights):.6f}")

# Show path statistics
maxops = weight_results['path_data']['maxop'][:, 0]
print(f"\nPath statistics:")
print(f"Order parameter range: {np.min(maxops):.6f} to {np.max(maxops):.6f}")
print(f"Paths crossing each interface:")
for i, intf in enumerate(weight_results['interfaces']):
    n_crossing = np.sum(maxops >= intf)
    print(f"  λ_{i} = {intf:.6f}: {n_crossing} paths")

# Compute weight matrices
print("\n=== STEP 2: COMPUTING WEIGHT MATRICES ===")
weight_matrices_results = compute_weight_matrices_weights(weight_results, tr=True)
w_path = weight_matrices_results['weight_matrix_3d']
w_path_2d = weight_matrices_results['weight_matrix_2d']
print("Weight matrices computed successfully!")

=== STEP 1: CALCULATING INFRETIS WEIGHTS ===
Found ptype information in column 4
Processed 9593 non-zero paths
Interfaces: [-0.1, 0.0, 0.1, 0.2, 0.3]
Path type information detected:
  Forward paths (dir=1): 5937
  Backward paths (dir=-1): 3656
  Other paths (dir=0): 0

Weight calculation summary:
Number of interfaces: 5
Interfaces: [-0.1, 0.0, 0.1, 0.2, 0.3]
Number of paths processed: 9593

Path statistics:
Order parameter range: -0.099990 to 0.300710
Paths crossing each interface:
  λ_0 = -0.100000: 9593 paths
  λ_1 = 0.000000: 8432 paths
  λ_2 = 0.100000: 7523 paths
  λ_3 = 0.200000: 6532 paths
  λ_4 = 0.300000: 5598 paths

=== STEP 2: COMPUTING WEIGHT MATRICES ===
Computing weight matrices for 9593 paths
Structure: Dictionary of 2D matrices [ensemble_idx][start_interface, end_interface]
Dimensions: 5 ensembles x 5 interfaces x 5 interfaces
Following original istar_analysis.py logic with tr=True
Using ptype information with direction for istar_analysis-style computation
Processed 959

/tmp/ipykernel_184690/231872884.py:165: RuntimeWarning: invalid value encountered in divide
  w = D["path_f"] / D["path_w"]
/tmp/ipykernel_184690/231872884.py:169: RuntimeWarning: invalid value encountered in divide
  w = w / np.sum(w, axis=0) * np.sum(D["path_f"], axis=0)


In [ ]:
# Analyze using tistools
print("\n=== STEP 3: ANALYZING WEIGHT MATRICES ===")
from tistools import get_transition_probs_weights, construct_M_istar, global_pcross_msm_star, ploc_memory

# Construct transition matrix using iSTAR approach
p, q = get_transition_probs_weights(weight_matrices_results['weight_matrix_3d'])
M_istar = construct_M_istar(p, 2*len(weight_matrices_results['interfaces']), len(weight_matrices_results['interfaces']))
print("iSTAR transition matrix constructed successfully!")
# Calculate global crossing probability using iSTAR
z1, z2, y1, y2 = global_pcross_istar(M_istar, doprint=True)
istar_global_pcross = y1[0] if y1.shape[0] > 1 else None
print(f"Global crossing probability (iSTAR): {istar_global_pcross}")


=== STEP 3: ANALYZING WEIGHT MATRICES ===

Intermediate transition probabilities (q matrix):
[[1.     0.7327 0.882  0.8703 0.8623]
 [1.     0.     1.     0.8443 0.8796]
 [0.8173 1.     0.     1.     0.7893]
 [0.8641 0.8972 1.     0.     1.    ]
 [0.8451 0.9372 0.9375 1.     0.    ]]

Final transition probabilities (p matrix):
[[0.2673 0.0864 0.0838 0.0774 0.485 ]
 [1.     0.     0.1557 0.1017 0.7426]
 [0.8173 0.1827 0.     0.2107 0.7893]
 [0.7752 0.122  0.1028 0.     1.    ]
 [0.7425 0.1361 0.0589 0.0625 0.    ]]

Local crossing probabilities computed successfully
iSTAR transition matrix constructed successfully!
iSTAR calculation details:
Intermediate states matrix Mp shape: (7, 7)
Boundary transition matrix D shape: (7, 2)
z1 (boundary conditions): [0 1]
z2 (intermediate solutions): [0.51096877 0.78324741 0.82732071 1.         0.         0.14307129
 0.18056915]
y1 (boundary probabilities): [0.51096877 0.        ]
Global crossing probability: 0.0
Global crossing probability (iSTAR): 

## Step 2: Compute Transition Probabilities

Next, we calculate the transition probabilities between interfaces using the infretis weights. This forms the basis for the iSTAR transition matrix.

In [ ]:
# Calculate transition probabilities
print("=== STEP 2: COMPUTING TRANSITION PROBABILITIES ===")

# Try both methods to compare
print("\nMethod 1: Standard iSTAR approach")
try:
    P_istar = 
    print("✓ iSTAR transition probabilities calculated successfully")
except Exception as e:
    print(f"✗ Error in iSTAR method: {e}")
    P_istar = None

print("\nMethod 2: Simplified direct counting approach")
try:
    P_simple = compute_simplified_transition_probabilities(weight_results)
    print("✓ Simplified transition probabilities calculated successfully")
    print("\nSimplified transition probability matrix:")
    print("Rows: starting interface, Columns: ending interface")
    for i in range(P_simple.shape[0]):
        print(f"Interface {i}: {P_simple[i, :]}")
except Exception as e:
    print(f"✗ Error in simplified method: {e}")
    P_simple = None

# Use the method that worked
if P_simple is not None:
    P = P_simple
    method_used = "Simplified"
elif P_istar is not None:
    P = P_istar  
    method_used = "iSTAR"
else:
    print("✗ Both methods failed!")
    P = None

if P is not None:
    print(f"\nUsing {method_used} method for further analysis")
    print(f"Transition matrix properties:")
    print(f"- Shape: {P.shape}")
    print(f"- Row sums: {[np.sum(P[i, :]) for i in range(P.shape[0])]}")
    print(f"- Determinant: {np.linalg.det(P):.6f}")
    
    # Check if matrix is properly stochastic
    row_sums = np.sum(P, axis=1)
    is_stochastic = np.allclose(row_sums, 1.0)
    print(f"- Is stochastic: {is_stochastic}")
    if not is_stochastic:
        print(f"  Row sums: {row_sums}")
else:
    print("Cannot proceed without transition probabilities!")

## Step 3: Construct iSTAR Transition Matrix and Calculate Global Pcross

Now we construct the full iSTAR transition matrix and calculate the global crossing probability using the Markov model approach.

In [ ]:
# Construct iSTAR transition matrix and calculate global crossing probability
print("=== STEP 3: iSTAR MATRIX CONSTRUCTION AND GLOBAL PCROSS ===")

if P is not None:
    N = len(weight_results['interfaces'])
    print(f"Number of interfaces: {N}")
    
    # Construct the iSTAR transition matrix
    try:
        M = construct_istar_transition_matrix(P, N)
        print("✓ iSTAR transition matrix constructed successfully")
        
        # Display matrix properties
        print(f"\nMatrix M properties:")
        print(f"- Shape: {M.shape}")
        print(f"- Non-zero entries: {np.count_nonzero(M)}")
        print(f"- Matrix preview (first 5x5):")
        print(M[:5, :5])
        
    except Exception as e:
        print(f"✗ Error constructing iSTAR matrix: {e}")
        M = None
    
    # Calculate global crossing probability
    if M is not None:
        try:
            print("\nCalculating global crossing probability...")
            z1, z2, y1, y2 = global_pcross_istar(M, doprint=True)
            
            if y1 is not None and y1.shape[0] > 1:
                global_pcross = y1[1, 0]  # Probability to reach state -1 before state 0
                print(f"\n✓ Global crossing probability calculated: {global_pcross:.8f}")
                
                # Store results for comparison
                istar_results = {
                    'transition_matrix': M,
                    'global_pcross': global_pcross,
                    'solution_vectors': (z1, z2, y1, y2)
                }
            else:
                print("✗ Could not extract global crossing probability")
                global_pcross = None
                istar_results = None
                
        except Exception as e:
            print(f"✗ Error calculating global crossing probability: {e}")
            global_pcross = None
            istar_results = None
    else:
        print("Cannot calculate global crossing probability without transition matrix")
        global_pcross = None
        istar_results = None
else:
    print("Cannot proceed without transition probabilities!")
    global_pcross = None
    istar_results = None

# Summary
if global_pcross is not None:
    print(f"\n{'='*50}")
    print(f"ISTAR ANALYSIS SUMMARY")
    print(f"{'='*50}")
    print(f"Global Crossing Probability: {global_pcross:.8f}")
    print(f"Method used: {method_used} transition probabilities")
    print(f"Number of interfaces: {N}")
    print(f"Interface positions: {weight_results['interfaces']}")
else:
    print("\n✗ iSTAR analysis failed - could not calculate global crossing probability")

## Step 4: Calculate Binless Crossing Probability

For comparison, we also calculate the binless crossing probability directly from the weighted path data. This provides a complementary view of the crossing behavior.

In [ ]:
# Calculate binless crossing probability
print("=== STEP 4: CALCULATING BINLESS CROSSING PROBABILITY ===")

try:
    x_binless, y_binless = calculate_binless_pcross(weight_results)
    print("✓ Binless crossing probability calculated successfully")
    
    # Show some key values
    interfaces = weight_results['interfaces']
    print(f"\nBinless crossing probability at interfaces:")
    for i, intf in enumerate(interfaces):
        # Find closest point in binless curve
        idx = np.argmin(np.abs(x_binless - intf))
        pcross_at_intf = y_binless[idx]
        print(f"  λ_{i} = {intf:.6f}: P_cross ≈ {pcross_at_intf:.6f}")
    
    binless_results = (x_binless, y_binless)
    
except Exception as e:
    print(f"✗ Error calculating binless crossing probability: {e}")
    binless_results = None

# Prepare results for comparison plotting
if binless_results is not None:
    plot_results = {
        'binless': binless_results,
        'interfaces': weight_results['interfaces']
    }
    
    if global_pcross is not None:
        plot_results['istar_global'] = global_pcross
        
        # Compare global iSTAR result with binless at final interface
        final_intf_idx = np.argmin(np.abs(x_binless - interfaces[-1]))
        binless_final = y_binless[final_intf_idx]
        
        print(f"\n{'='*50}")
        print(f"COMPARISON SUMMARY")
        print(f"{'='*50}")
        print(f"iSTAR Global P_cross:     {global_pcross:.8f}")
        print(f"Binless at final λ:       {binless_final:.8f}")
        print(f"Ratio (iSTAR/Binless):    {global_pcross/binless_final:.6f}")
        print(f"Absolute difference:      {abs(global_pcross - binless_final):.8f}")
    
    print("\n✓ Ready for visualization")
else:
    print("✗ Cannot prepare comparison plots without binless results")

## Step 5: Visualization and Analysis

Finally, we visualize the results and compare the different methods for calculating the crossing probability.

In [ ]:
# Visualization and final analysis
print("=== STEP 5: VISUALIZATION AND ANALYSIS ===")

if 'plot_results' in locals() and plot_results:
    # Plot the comparison
    print("Creating comparison plot...")
    plot_pcross_comparison(plot_results, 
                          title="iSTAR vs Binless Crossing Probability (infretis weights)")
    
    # Additional analysis plots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Plot 1: Transition matrix heatmap (if available)
    if istar_results is not None and 'transition_matrix' in istar_results:
        M = istar_results['transition_matrix']
        im1 = axes[0, 0].imshow(M, cmap='Blues', aspect='auto')
        axes[0, 0].set_title('iSTAR Transition Matrix M')
        axes[0, 0].set_xlabel('To State')
        axes[0, 0].set_ylabel('From State')
        plt.colorbar(im1, ax=axes[0, 0])
    else:
        axes[0, 0].text(0.5, 0.5, 'Transition Matrix\nNot Available', 
                       ha='center', va='center', transform=axes[0, 0].transAxes)
        axes[0, 0].set_title('iSTAR Transition Matrix M')
    
    # Plot 2: Local crossing probabilities
    interfaces = weight_results['interfaces']
    ploc = weight_results['ploc_wham']
    axes[0, 1].semilogy(range(len(interfaces)), ploc, 'ro-', linewidth=2, markersize=8)
    axes[0, 1].set_title('Local Crossing Probabilities (WHAM)')
    axes[0, 1].set_xlabel('Interface Index')
    axes[0, 1].set_ylabel('P_loc')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Weight distribution
    weights = weight_results['unbiased_weights']
    axes[1, 0].hist(np.log10(weights[weights > 0]), bins=20, alpha=0.7, color='green')
    axes[1, 0].set_title('Distribution of Path Weights')
    axes[1, 0].set_xlabel('log₁₀(Weight)')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Path statistics
    maxops = weight_results['path_data']['maxop'][:, 0]
    axes[1, 1].scatter(maxops, weights, alpha=0.6, s=20)
    axes[1, 1].set_title('Path Weights vs Maximum Order Parameter')
    axes[1, 1].set_xlabel('Maximum Order Parameter')
    axes[1, 1].set_ylabel('Weight')
    axes[1, 1].set_yscale('log')
    
    # Add interface lines
    for intf in interfaces:
        axes[1, 1].axvline(intf, color='red', linestyle='--', alpha=0.7)
    
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✓ Visualization complete")
    
else:
    print("✗ Cannot create plots - missing results data")

# Final summary
print(f"\n{'='*60}")
print(f"FINAL WORKFLOW SUMMARY")
print(f"{'='*60}")

if global_pcross is not None:
    print(f"✓ iSTAR Analysis Successful")
    print(f"  Global Crossing Probability: {global_pcross:.8e}")
    print(f"  Method: iSTAR with {method_used} transition probabilities")
else:
    print(f"✗ iSTAR Analysis Failed")

if binless_results is not None:
    print(f"✓ Binless Analysis Successful")
    print(f"  Points calculated: {len(binless_results[0])}")
    print(f"  Order parameter range: {binless_results[0].min():.3f} to {binless_results[0].max():.3f}")
else:
    print(f"✗ Binless Analysis Failed")

print(f"\nKey Improvements over Old Workflow:")
print(f"• Uses proper infretis weights instead of simple path counts")
print(f"• Implements WHAM-based local crossing probabilities")
print(f"• Provides both iSTAR and binless methods for comparison")
print(f"• Includes comprehensive error handling and diagnostics")

print(f"\n{'='*60}")

## Key Improvements and Usage Notes

### Major Changes from Old Workflow:

1. **Infretis Weight Integration**: The biggest change is using proper infretis weights (`path_f / path_w`) instead of simple path counting. This provides statistically correct weighting of paths.

2. **WHAM-based Local Probabilities**: The calculation of local crossing probabilities now uses the WHAM methodology to properly combine data from different ensembles.

3. **Robust Error Handling**: The workflow includes comprehensive error checking and fallback methods.

4. **Multiple Methods**: Provides both iSTAR Markov model approach and direct binless calculation for comparison.

### Understanding the Results:

- **Global Pcross (iSTAR)**: Single value representing the overall transition probability calculated using the Markov state model
- **Binless Pcross**: Continuous curve showing crossing probability as a function of order parameter
- **Local Probabilities**: WHAM-weighted probabilities for transitions between adjacent interfaces
- **Transition Matrix**: Full Markov model capturing the interface-to-interface dynamics

### Usage for Different Systems:

To use this workflow for your own system:
1. Update `DATA_DIR`, `TOML_FILE`, and `DATA_FILE` paths
2. Adjust `NSKIP` if you want to exclude initial equilibration
3. The interfaces are automatically read from the TOML file
4. All weight calculations are handled automatically

### Validation:

The workflow validates results by:
- Comparing iSTAR global result with binless calculation
- Checking matrix properties (stochastic, determinant)
- Monitoring weight distributions and statistics
- Providing comprehensive diagnostic output